<a href="https://colab.research.google.com/github/chizukeiku/PlantCanopyTemperature/blob/main/PlantCanopyMask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/chizukeiku/PlantCanopyTemperature.git # git clone
%cd PlantCanopyTemperature

%pip install plantcv # install library
%pip install altair

In [ ]:
!ls # list files in directory

In [ ]:
# modules needed
import os
import cv2 as cv
import numpy as np
import pandas as pd
from plantcv.plantcv import print_image
from plantcv.plantcv import plot_image
from plantcv.plantcv import fatal_error
from plantcv.plantcv import params
from plantcv.plantcv import outputs
from plotnine import ggplot, geom_label, aes, geom_line
from matplotlib import pyplot as plt
import matplotlib
from plantcv import plantcv as pcv

In [ ]:
# raw plant thermal image here
# Img = cv.imread('change_file_here.PNG')

Img = cv.imread('plant_image/plant_image (1).PNG')
Img_gry = cv.cvtColor(Img, cv.COLOR_BGR2GRAY)
plt.imshow(Img_gry, cmap='gray')
plt.show()

In [ ]:
# masking

# set figure size for the notebook
matplotlib.rcParams['figure.figsize'] = [8,8]

class options:
    def __init__(self):
        self.image = "plant_image/plant_image (1).jpg" # plant RGB image path (mask)
        self.debug = "plot"
        self.writeimg = False
        self.result = "result.txt"
        self.outdir = "."
args = options()
pcv.params.outdir = args.outdir
pcv.params.debug = args.debug

# increase text size in image plots
pcv.params.text_size = 20
pcv.params.text_thickness = 10
# read the image
img, path, filename = pcv.readimage(filename=args.image)

# change image color
image = pcv.rgb2gray_lab(rgb_img=img, channel="l")

# image segmentation with binary threshold
# mask_method = pcv.threshold.binary(gray_img=l, threshold=insert_desired_value, max_value=256, object_type="dark")
mask_method = pcv.threshold.binary(gray_img=l, threshold=70, max_value=256, object_type="dark")

# fill in some holes in segmented image
# fill_mask = pcv.fill(bin_img=mask_method, size=insert_desired_value)
fill_mask = pcv.fill(bin_img=mask_method, size=7)

id_objects, obj_hierarchy = pcv.find_objects(img=img, mask=fill_mask)

# plant region of interest
# roi_circle, roi_hier_circle = pcv.roi.circle(img=img, x=insert_desired_value, y=insert_desired_value, r=insert_desired_value)
roi_circle, roi_hier_circle = pcv.roi.circle(img=img, x=67, y=45, r=35)

roi_objects, hierarchy, kept_mask, obj_area = pcv.roi_objects(img=img, roi_contour=roi_circle, roi_hierarchy=roi_hier_circle, object_contour=id_objects, obj_hierarchy=obj_hierarchy, roi_type='cutto')
mask = pcv.fill(bin_img=kept_mask, size=500)

In [ ]:
# clip mask
mask_thermal = pcv.apply_mask(img=mask, mask=mask, mask_color="black")
plt.imshow(mask_thermal)
plt.show()

In [ ]:
# export mask
pcv.print_image(mask_thermal, 'mask.tiff')